In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import random
import pandas as pd
import numpy as np
from pathlib import Path

In [3]:
# fix seeds, make everything reproducible, etc (at least try to).
SEED = 42
np.random.seed(SEED)
random.seed(SEED)

DATADIR = "../data"

MAX_WORDS = 4

In [4]:
# !pip install aicrowd-cli
%load_ext aicrowd.magic

In [5]:
%aicrowd login

API Key valid
Gitlab oauth token invalid or absent.
It is highly recommended to simply run `aicrowd login` without passing the API Key.
Saved details successfully!


In [6]:
import re
if Path(DATADIR).exists():
  !rm -rf $DATADIR
!mkdir $DATADIR
%aicrowd ds dl -c htrec-2022 -o $DATADIR

train.csv:   0%|          | 0.00/395k [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/45.5k [00:00<?, ?B/s]

In [7]:
import pywer
train_df = pd.read_csv( f"{DATADIR}/train.csv")
test_df = pd.read_csv(f"{DATADIR}/test.csv")

word_regex = re.compile("\W+")
word_regex2 = re.compile("(\W+)")

train_df.head()

,HUMAN_TRANSCRIPTION,SYSTEM_TRANSCRIPTION,CENTURY,IMAGE_PATH,TEXT_LINE_NUM
0,ἐγγινομένα πάθη μὴ σβεννύντες ἀλλὰ τῆ εκλύσει,ἐγγενομεναπαδημησμεννωτες ἀλλατῆε κλησει,11,1 Bodleian-Library-MS-Barocci-102_00157_fol-75...,1
1,τοῦ βίου τοῦ καθ ΄ εαυτοὺς πολλὰ γίνεσθαι συγχ...,του β ου του καλεαυτοὺς πολλαγινεσθαι συγχωρ όν,11,1 Bodleian-Library-MS-Barocci-102_00157_fol-75...,2
2,τες ἐμπυρίζουσι τὸν ἀμπελῶνα ἀλλὰ καὶ ὁ διὰ,τες εμπυριζου σιμαμπελῶνα ἀλλακαι ὅδξα,11,1 Bodleian-Library-MS-Barocci-102_00157_fol-75...,3
3,τῆς ἡδεῖας πλεονεξίας πολλοὺς εἰς τὴν τῶν ἀλλ,της ἐδίας πλσον ἐξιας πολλους ἐις τὴν τῶν ἀλ,11,1 Bodleian-Library-MS-Barocci-102_00157_fol-75...,4
4,οτρίων ἐπιθυμίαν προκαλούμενος ἐμπυρί,λοτρλων ἐπιθυμιαν προκαλουμένος ἐμπυρι,11,1 Bodleian-Library-MS-Barocci-102_00157_fol-75...,5


In [8]:
# !pip install nltk

In [9]:
for lib_dir in ["..", "../src"]:
    if not lib_dir in sys.path:
        sys.path.append(lib_dir)
from lm_utils import *

In [10]:
language_model = make_lm(train_df.HUMAN_TRANSCRIPTION.sum())

In [11]:
from tqdm.auto import tqdm
from datastruct import *
from common import *
from space_fixer import SpaceFixer

# for ht_line, mt_line in tqdm(train_df[["HUMAN_TRANSCRIPTION", "SYSTEM_TRANSCRIPTION"]].values[7:]):
#     ht_words = word_regex.split(ht_line)
#     mt_words = word_regex.split(mt_line)
#     #words_ = [remove_cap(word) for word in words]
#     #vocab.add_sentence(words)
#     dmatrix = build_path_matrix(mt_words, vocabs)
#     finished_paths = extract_paths(dmatrix)
#     for k in resplit_paths(finished_paths, mt_words):
#         variant = " ".join(k)
#         print(variant)
#     break
CUTOFF = 3

In [12]:
def extract_spaces_dict(x):
    result = dict()
    last_index = 0
    for w in word_regex2.split(x):
        if not word_regex.match(w):
            last_index += 1
        else:
            result[last_index] = w
    if not last_index in result:
        result[last_index] = ""
    return result
    

In [13]:

train_df["SYSTEM_TRANSCRIPTION_raw"] = train_df.SYSTEM_TRANSCRIPTION
train_df["SYSTEM_TRANSCRIPTION"] = train_df.SYSTEM_TRANSCRIPTION.apply(lambda x: lmr(x, lm=language_model))

ht_sequences_train = train_df.HUMAN_TRANSCRIPTION.apply(lambda x: word_regex.split(x)).values
mt_sequences_train = train_df.SYSTEM_TRANSCRIPTION.apply(lambda x: word_regex.split(x)).values
mt_spaces_train = train_df.SYSTEM_TRANSCRIPTION.apply(extract_spaces_dict).values

mt_texts_train = train_df.SYSTEM_TRANSCRIPTION.values
fixer = SpaceFixer(MAX_WORDS)
fixer.fill(ht_sequences_train)

  0%|          | 0/1875 [00:00<?, ?it/s]

In [14]:
additional_corpus = pd.read_csv("../mgc.csv")
additional_corpus.head()
additional_corpus.loc[additional_corpus.text.isnull()]


,filename,text,genre
18,Greek_Medieval_Corpus/Ποιητικά-Λογοτεχνικά/poe...,NaN,Ποιητικά-Λογοτεχνικά/poetry
25,Greek_Medieval_Corpus/Ποιητικά-Λογοτεχνικά/poe...,NaN,Ποιητικά-Λογοτεχνικά/poetry


In [16]:
sep_regex = re.compile("<[^>]+>")

sentence_regex = re.compile("[.!?]")
def split_to_sentences(text):
    if len(set(sep_regex.findall(text)) | {"<NEWLINE>", "<NEWPARAGRAPH>"}) > 2:
        return
    for text_block in sep_regex.split(text):
        for sentences in sentence_regex.split(text_block):
            for sentence in sentences.split("\n"):
                words = [w for w in word_regex.split(sentence) if len(w) > 0]
                if len(words) < 1:
                    continue
                if len(words) > 20:
                    continue
                has_short_words = np.any([len(w1) <= 2 and len(w2) <= 2 for w1, w2 in zip(words, words[1:])])
                if has_short_words:
                    continue
                yield words
additional_corpus = additional_corpus.loc[~additional_corpus.text.isnull()]

# set([x for xs in additional_corpus.text.apply(lambda x: sep_regex.findall(x)).values for x in xs])
additional_corpus['cleaned_sentences'] = additional_corpus.text.apply(lambda x: list(split_to_sentences(x)))
#additional_corpus.text.isnull().mean()
flatten_sentences = [sentence for sentences in additional_corpus.cleaned_sentences.values for sentence in sentences if len(sentence) > 0]

fixer.fill(flatten_sentences)

  0%|          | 0/44666 [00:00<?, ?it/s]

In [18]:
# additional_corpus.text.apply(lambda x: set(sep_regex.findall(x)) )
len(fixer.vocabs[1].words)

71998

In [19]:
flatten_sentences[:3]

[['ΜΕΤΑ', 'ΓΟΥΝ', 'ΑΛΛΑ', 'ΤΑ', 'ΠΟΛΛΑ', 'ΤΩΝ', 'ΕΡΩΤΟΧΑΡΙΤΩΝ'],
 ['ΟΣΑ', 'ΜΑΝΘΑΝΕΙ', 'ΦΥΣΙΚΩΣ', 'ΕΡΩΤΙΚΗ', 'ΚΑΡΔΙΑ'],
 ['ΕΣΕΒΗΣΑΝ', 'ΕΙΣ', 'ΤΟ', 'ΛΟΥΤΡΟΝ', 'ΕΛΟΥΣΘΗΣΑΝ', 'ΕΚΕΙΝΟΙ']]

In [21]:
from common import correct_sigmas_in_word
from common import fix_accent_diphthong

def join_if_tuple(s):
    if isinstance(s, str):
        return s
    return " ".join([(x) for x in s])

def postprocess_spaces(spaces_after):
    #return spaces_after
    if len(spaces_after) < 2:
        return spaces_after
    if spaces_after[-1]!=".":
        spaces_after[-1] = ''
    return spaces_after


corrected_texts = []
# mt_sequences_train = mt_sequences_train[-1:]
# mt_spaces_train = mt_spaces_train[-1:]
# mt_texts_train = mt_texts_train[-1:]
# ht_sequences_train = ht_sequences_train[-1:]
corrected_count = 0
corrected_raw = []
train_iter = zip(
    mt_sequences_train, 
    mt_spaces_train, 
    mt_texts_train, 
    ht_sequences_train)
for i, (mt_words, line_spaces, mt_orig, ht_orig) in enumerate(tqdm(train_iter, total=len(mt_sequences_train))):
    # print(line_spaces)
    # print(mt_words)
    # mt_orig = " ".join(mt_words)
    best = mt_orig
    # temporary disable the following lines
    replacements_ = [
        [(w, s) for w, s in zip(mt_split, postprocess_spaces(spaces_after))]
        for mt_split, refs, spaces_after in fixer.split_words(mt_words, line_spaces, cutoff=CUTOFF)
    ]
    replacements = [
        [join_if_tuple(w) + s for w, s in replacement]
        for replacement in replacements_
    ]
    replacements = ["".join(words) for words in replacements]

    #replacements = [
    #    ("".join([w + s for w, s in zip(mt_split, spaces_after)]), spaces_after) 
    #    
    #]
    # replacements, spaces_after = list(zip(*replacements))
    # replacements = list(replacements)
    if len(replacements) > 2:
        print(i, len(replacements))
        print("mt orig", mt_orig)
        print(replacements)
        # print(spaces_after)
        print(line_spaces)
        # break
    
    best, best_index = lm_score(mt_orig, replacements,
        lm=language_model, return_index=True
        )
    if best_index > 0:
        corrected_count += 1
        corrected_raw.append(replacements_[best_index - 1])
    else:
        corrected_raw.append(mt_orig)
    corrected_texts.append(best)

    # break

  0%|          | 0/1875 [00:00<?, ?it/s]

175 3
mt orig της δόξης του ιματισμου εφαιρειται μεντοι
['της δόξης του ιματισμου εφαιρει ται μεντοι', 'της δόξης του ιματισμου ε φαιρειται μεντοι', 'της δόξης του ιματισμου εφαιρειται μεντοι']
{1: ' ', 2: ' ', 3: ' ', 4: ' ', 5: ' ', 6: ''}
407 3
mt orig εωρὸς ὸν οὐκδίασον ὐ καίνηρ χριρ
['εωρὸς ὸν οὐ κ δίασον ὐ καίνηρ χριρ', 'εωρὸς ὸν οὐκ δίασ ο ν ὐ καίνηρ χριρ', 'εωρὸς ὸν οὐ κ δία σον ὐ καίνηρ χριρ']
{1: ' ', 2: ' ', 3: ' ', 4: ' ', 5: ' ', 6: ''}
703 4
mt orig δων δὲοις ότι επισυντρες
['δων δὲ οις ότι επι συντρες', 'δων δὲ ο ις ότι επι συντρες', 'δων δὲ οις ότι επισυν τρες', 'δων δὲ ο ις ότι επισυν τρες']
{1: ' ', 2: ' ', 3: ' ', 4: ''}
870 4
mt orig ἀλλδκ εταυτνησιν ἀλλήδε κα ὼν
['ἀλλδκ ετ αυτνησιν ἀλλή δε κα ὼν', 'ἀλλδκ ετα υ τνησιν ἀλλή δε κα ὼν', 'ἀλλδκ ε ταυτν ησιν ἀλλή δε κα ὼν', 'ἀλλδκ ε ταυτ νησιν ἀλλή δε κα ὼν']
{1: ' ', 2: ' ', 3: ' ', 4: ' ', 5: ''}
1034 3
mt orig μηρλόντον νεαν ν δυσιμενοις τὸν
['μηρλόντον νεαν ν δυσιμενοις τὸν', 'μηρλόντον νεαν ν δυσιμ ενοις τὸν', 'μηρλ

In [22]:
def clean_tuple(s):
    if isinstance(s, str):
        return s
    if isinstance(s, tuple) and len(s) == 1:
        return s[0]
    return s
def split_to_texts(seq):
    prefix = []
    for w, s in seq:
        if isinstance(w, tuple):
            if len(prefix) > 0:
                prefix = [w+s for w, s in prefix]
                yield "".join(prefix)
                prefix = []
            yield [(w, s)]
        else:
            prefix.append((w, s))
    if len(prefix) > 0:
        prefix = [w+s for w, s in prefix]
        yield "".join(prefix)

from greedy import greedy_correction, greedy_correction_one
greedy_corrected_texts = []
for text_seq in tqdm(corrected_raw):
    if isinstance(text_seq, str):
        text = greedy_correction_one(text_seq, fixer)
        greedy_corrected_texts.append(text)
    else:
        text_seq = [(clean_tuple(w), s) for w, s in text_seq]
        # break
        all_fragments = []
        for fragment in split_to_texts(text_seq):
            if isinstance(fragment, str):
                new_fragments = greedy_correction_one(fragment, fixer)
                all_fragments += new_fragments
            else:
                all_fragments += fragment
        greedy_corrected_texts.append(all_fragments)

    


  0%|          | 0/1875 [00:00<?, ?it/s]

╭──────────────────────────── Traceback (most recent call last) ────────────────────────────╮
│                                                                                           │
│ /var/folders/h6/ssrt88hx24gg57sp1hgv1ftm0000gn/T/ipykernel_95437/2363169651.py:34 in      │
│ <module>                                                                                  │
│                                                                                           │
│ [Errno 2] No such file or directory:                                                      │
│ '/var/folders/h6/ssrt88hx24gg57sp1hgv1ftm0000gn/T/ipykernel_95437/2363169651.py'          │
│ /Users/lacemaker/github/htrec-2022/notebooks/../src/greedy.py:110 in                      │
│ greedy_correction_one                                                                     │
│                                                                                           │
│   107 │   │   │   # print("fragment", fragment)                                           │
│   108 │   │   │   new_fragments += fragment                                               │
│   109 │   │   else:                                                                       │
│ ❱ 110 │   │   │   fixed_fragment = fix_spaces(fragment, fixer, max_words=max_words)       │
│   111 │   │   │   new_fragments+= fixed_fragment                                          │
│   112 │   # fragments = [join_if_tuple(w) + s for w, s in new_fragments]                  │
│   113 │   # text = "".join(fragments)                                                     │
│                                                                                           │
│ /Users/lacemaker/github/htrec-2022/notebooks/../src/greedy.py:68 in fix_spaces            │
│                                                                                           │
│    65 │   │   │   mt_word = Word([w for w, s in subseq])                                  │
│    66 │   │   │   if len(mt_word) == 0:                                                   │
│    67 │   │   │   │   break                                                               │
│ ❱  68 │   │   │   ref_word = find_in_fixer(mt_word, fixer)                                │
│    69 │   │   │   if ref_word is None:                                                    │
│    70 │   │   │   │   continue                                                            │
│    71 │   │   │   alignment_result = mt_word.distance_to(ref_word)                        │
│                                                                                           │
│ /Users/lacemaker/github/htrec-2022/notebooks/../src/greedy.py:42 in find_in_fixer         │
│                                                                                           │
│    39 │   │   mt_word = Word(mt_word)                                                     │
│    40 │   N = max(fixer.vocabs)                                                           │
│    41 │   for i in range(N):                                                              │
│ ❱  42 │   │   for ref_word, tree_dist in fixer.vocabs[i + 1].find_closest(mt_word.sequenc │
│    43 │   │   │   if tree_dist != 0:                                                      │
│    44 │   │   │   │   continue                                                            │
│    45 │   │   │   alignment_result = mt_word.distance_to(ref_word)                        │
│                                                                                           │
│ /Users/lacemaker/github/htrec-2022/notebooks/../src/datastruct.py:314 in find_closest     │
│                                                                                           │
│   311 │   │   else:                                                                       │
│   312 │   │   │   word = Word(query_str)                                                  │
│   313 │   │   query_vector = self.bow2vector(word.

In [23]:
greedy_corrected_texts = [
    "".join([join_if_tuple(w) + s for w, s in text])
    for text in greedy_corrected_texts
]

In [24]:
# # for mt_split, refs, spaces_after in fixer.split_words(mt_words, line_spaces, cutoff=5):
# #     print(mt_split, refs, spaces_after)
# # # print(ht_orig)
# # replacements
# # greedy_correction(mt_sequences_train, fixer)
# from greedy import greedy_correction
# splitted_texts = [word_regex2.split(text) for text in corrected_texts]
# corrected_texts = []
# corrected_count = 0
# for corrected_seq in greedy_correction(splitted_texts, fixer):
#     text = [join_if_tuple(w) + s for w, s in corrected_seq]
#     text = "".join(text)
#     corrected_texts.append(text)
#     if text != mt_orig:
#         corrected_count += 1
# corrected_count


In [25]:
unicodedata.name("σ"), unicodedata.name("ς")
print(corrected_count)
# fixer.split_matrix
# best, is_corrected = lm_score(mt_orig, replacements, lm=language_model, return_corrected=True)
# best, ht_orig
count = np.sum([mt_orig!= corrected for mt_orig, corrected in zip(mt_texts_train, greedy_corrected_texts)])
print(count)
greedy_corrected_texts[:3]

606
487


['ἐγγενομεναπαδημησμεννωτες ἀλλατῆε κλησει',
 'του β ου του καλ εαυτοὺς πολλα γινεσθαι συγχωρ όν',
 'τες εμπυριζου σιμαμπελῶνα ἀλλα και ὅδξα']

In [26]:
# K = 6
# train_iter = zip(
#     mt_sequences_train[K:],
#     mt_spaces_train[K:],
#     mt_texts_train[K:],
#     ht_sequences_train[K:]
# )

# for i, (mt_words, line_spaces, mt_orig, ht_orig) in enumerate(tqdm(train_iter, total=len(mt_sequences_train[K:]))):
#     break
# print("mt_original: ", mt_words)
# print("ht:", ht_orig)
# for mt_split, refs, spaces_after in fixer.split_words(mt_words, line_spaces, cutoff=0):
#     print(mt_split)



In [27]:
from metrics import compute_metrics

In [54]:
from common import fix_accents
ht_texts = train_df.HUMAN_TRANSCRIPTION.values
mt_texts = train_df.SYSTEM_TRANSCRIPTION_raw.values
ct = [lmr(t, lm=language_model) for t in greedy_corrected_texts]
ct2 = [fix_accent_diphthong(t) for t in ct]
ct3 = [fix_accents(t) for t in ct2]
# ct2 = [lmr(t, word=" δ ", replacements=["δ ", " δ", " "], lm=language_model) for t in ct]
print("Unmodified:")
cerr_values_ht, cerr_values = compute_metrics(ht_texts, mt_texts, corrected_texts)
print("Corrected sigmas:")
cerr_values_ht1, cerr_values1 = compute_metrics(ht_texts, mt_texts, ct)
print("corrected deltas:")
cerr_values_ht2, cerr_values2 = compute_metrics(ht_texts, mt_texts, ct2)
print("accents:")
cerr_values_ht2, cerr_values2 = compute_metrics(ht_texts, mt_texts, ct3)

Unmodified:
Candidate CER: 33.326478596708895
Candidate CERR: 0.9285963805836962
Corrected sigmas:
Candidate CER: 37.41469122427831


╭──────────────────────────── Traceback (most recent call last) ────────────────────────────╮
│                                                                                           │
│ /var/folders/h6/ssrt88hx24gg57sp1hgv1ftm0000gn/T/ipykernel_95437/3258382041.py:11 in      │
│ <module>                                                                                  │
│                                                                                           │
│ [Errno 2] No such file or directory:                                                      │
│ '/var/folders/h6/ssrt88hx24gg57sp1hgv1ftm0000gn/T/ipykernel_95437/3258382041.py'          │
│ /Users/lacemaker/github/htrec-2022/notebooks/../src/metrics.py:13 in compute_metrics      │
│                                                                                           │
│   10   print(f"Candidate CER: {cer_values.mean()}")                                       │
│   11   # Computing the character error *reduction* rate (CERR)                            │
│   12   cer_ht_st = compute_cer_values(human_translations, system_translations)            │
│ ❱ 13   print(f"Candidate CERR: {(cer_ht_st - cer_values).mean()}")                        │
│   14   return cer_ht_st, cer_values                                                       │
╰───────────────────────────────────────────────────────────────────────────────────────────╯
ValueError: operands could not be broadcast together with shapes (1875,) (1532,)

In [56]:
diff_cer = (cerr_values_ht - cerr_values)
idx = np.argsort(diff_cer)
for i in idx[:5]:
    print(diff_cer[i])
    print(repr(ct[i]))
    print(repr(ht_texts[i]), repr(mt_texts[i]))

-8.57142857142857
'τις αρετῆς αναγίοιώ τον διά κμ ειναι'
'τῆς ἀρετῆς ἀναλλοίωτον διαμεῖναι. Ἡ' 'τις αρετῆς αναγίοιώτον διάκμειναι'
-5.555555555555557
'εἐργων τῆς δι και ουνος ὶ ειπης ψγ'
'ἔργων τῆς δικαιοσύνης. Μή εἴπῃς, γάρ' 'εἐργων τῆς δικαιουνος ὶ ειπης ψγ'
-5.555555555555557
'σιν μποιοξαν κὐ ταχὴν προ τάρ ηες αν'
'σοὶ ξυμπονῆσαι καὶ ταχὺν προσαρκέσαι' 'σιν μποιοξαν κὐ ταχὴν προτάρηες αν'
-5.555555555555557
'τοῦ δυσιν εσαιλόγος σξωην ἐκρ'
'τοῦ Θῦ γίνεται Λόγος, ὡς ζωή νεκροῖς' 'τοῦ δυσιν εσαιλόγος ς ξωην ἐκρ'
-5.263157894736835
'ὁ γι ότε α τις α τειλεισαρνλκαι εφυλαζεητόν'
'Ὁπότε ἀπέστειλε Σαοὺλ, καὶ ἐφύλαξε τὸν' 'ὁ γιότεατις α τειλεισαρνλκαι εφυλαζεητόν'


In [57]:
test_df["SYSTEM_TRANSCRIPTION_raw"] = test_df.SYSTEM_TRANSCRIPTION
test_df["SYSTEM_TRANSCRIPTION"] = test_df.SYSTEM_TRANSCRIPTION.apply(lambda x: lmr(x, lm=language_model))

In [58]:
mt_orig_test = test_df.SYSTEM_TRANSCRIPTION.values
mt_sequences_test = test_df.SYSTEM_TRANSCRIPTION.apply(lambda x: word_regex.split(x)).values
mt_spaces_test = test_df.SYSTEM_TRANSCRIPTION.apply(extract_spaces_dict).values
corrected_texts_test = []
corrected_raw_test = []
corrected_count_test = 0
for i, (mt_words, line_spaces, mt_orig) in enumerate(tqdm(zip(mt_sequences_test, mt_spaces_test, mt_orig_test), total=len(mt_sequences_test))):
    # print(i)
    # print(mt_words)
    # mt_orig = " ".join(mt_words)
    # temporary disable the following lines
    best = mt_orig
    # replacements = ["".join([w+s for w, s in zip(mt_split, spaces_after)]) for mt_split, refs, spaces_after in fixer.split_words(mt_words, mt_spaces)]
    replacements_ = [
        [(w, s) for w, s in zip(mt_split, postprocess_spaces(spaces_after))]
        for mt_split, refs, spaces_after in fixer.split_words(mt_words, line_spaces, cutoff=CUTOFF)
    ]
    replacements = [
        [join_if_tuple(w) + s for w, s in replacement]
        for replacement in replacements_
    ]
    replacements = ["".join(words) for words in replacements]
    best, best_index = lm_score(
        mt_orig, replacements,
        lm=language_model, return_index=True
    )
    if best_index > 0:
        corrected_count_test += 1
        corrected_raw_test.append(replacements_[best_index - 1])
    else:
        corrected_raw_test.append(mt_orig)

    # best = lm_score(mt_orig, replacements, lm=language_model)
    corrected_texts_test.append(best)

  0%|          | 0/338 [00:00<?, ?it/s]

In [59]:
greedy_corrected_texts_test = []
for text_seq in tqdm(corrected_raw_test):
    if isinstance(text_seq, str):
        text = greedy_correction_one(text_seq, fixer)
        greedy_corrected_texts_test.append(text)
    else:
        text_seq = [(clean_tuple(w), s) for w, s in text_seq]
        # break
        all_fragments = []
        for fragment in split_to_texts(text_seq):
            if isinstance(fragment, str):
                new_fragments = greedy_correction_one(fragment, fixer)
                all_fragments += new_fragments
            else:
                all_fragments += fragment
        greedy_corrected_texts_test.append(all_fragments)

  0%|          | 0/338 [00:00<?, ?it/s]

In [60]:
# greedy_corrected_texts_test
greedy_corrected_texts_test = [
    "".join([join_if_tuple(w) + s for w, s in text])
    for text in greedy_corrected_texts_test
]

In [64]:
corrected_texts_test1 = [lmr(t, lm=language_model) for t in greedy_corrected_texts_test]
print(np.sum([t1!= t for t1, t in zip(corrected_texts_test1, corrected_texts_test)]))
# corrected_texts_test2 = [
#     lmr(t, word=" δ ", replacements=["δ ", " δ", " "], lm=language_model)
#     for t in corrected_texts_test1
# ]
corrected_texts_test2 = [fix_accent_diphthong(t) for t in corrected_texts_test1]
print(np.sum([t1 != t for t1, t in zip(corrected_texts_test2, corrected_texts_test)]))
print(np.sum([t1 != t for t1, t in zip(corrected_texts_test2, mt_orig_test)]))
corrected_texts_test3 = [fix_accents(t) for t in corrected_texts_test2]
print(np.sum([t1 != t for t1, t in zip(corrected_texts_test3, corrected_texts_test2)]))

5
7
60
49


In [65]:
corrected_texts_test2[:4]

['ὑπρ τη συνευςδιξοεανδνυπορ ποοδυπρας',
 'ἢ μητακχεφιλοχορ εύτα δυ μπρο πρενπεμε',
 'συ σ κατεχισωμε ἐπι τελωτι',
 'κ ἀπαὐτελετία τὸν δεὲ τὸν σανἀαλισκόν']

In [66]:
submission = pd.DataFrame(
    zip(test_df.IMAGE_PATH, corrected_texts_test3),
    columns=["ImageID", "Transcriptions"]
)
submission.sample()

,ImageID,Transcriptions
110,105 Bodleian-Library-MS-Barocci-59_00085_fol-4...,τὲ πρμον ην τὴν αν θησιν τὴν σρ


In [67]:
submission.to_csv("submission.csv", index=False)

In [68]:
%aicrowd submission create -c htrec-2022 -f submission.csv

submission.csv ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.0% • 45.4/43.7 KB • 513.9 kB/s • 0:00:00

╭─────────────────────────╮                                     
                                    │ Successfully submitted! │                                     
                                    ╰─────────────────────────╯

                                          Important links                                           
┌──────────────────┬───────────────────────────────────────────────────────────────────────────────┐
│  This submission │ https://www.aicrowd.com/challenges/htrec-2022/submissions/191928              │
│                  │                                                                               │
│  All submissions │ https://www.aicrowd.com/challenges/htrec-2022/submissions?my_submissions=true │
│                  │                                                                               │
│      Leaderboard │ https://www.aicrowd.com/challenges/htrec-2022/leaderboards                    │
│                  │                                                                               │
│ Discussion forum │ https://discourse.aicrowd.com/c/htrec-2022                                    │
│                  │                                                                               │
│   Challenge page │ https://www.aicrowd.com/challenges/htrec-2022                                 │
└──────────────────┴───────────────────────────────────────────────────────────────────────────────┘

{'submission_id': 191928, 'created_at': '2022-07-02T02:38:31.675Z'}


In [ ]:
#test_spaces = test_df.SYSTEM_TRANSCRIPTION.apply(lambda x: [w for w in word_regex2.split(x) if word_regex.match(w)]).values
#train_spaces1 = train_df.SYSTEM_TRANSCRIPTION.apply(lambda x: [w for w in word_regex2.split(x) if word_regex.match(w)]).values
#train_spaces2 = train_df.HUMAN_TRANSCRIPTION.apply(lambda x: [w for w in word_regex2.split(x) if word_regex.match(w)]).values

In [ ]:
#from collections import Counter
#Counter([x for xs in test_spaces for xss in xs for x in xss])

In [ ]:
#Counter([x for xs in train_spaces1 for xss in xs for x in xss])

In [ ]:
# Counter([x for xs in train_spaces2 for xss in xs for x in xss])

In [152]:
# n_corrected = 0
# for corrected, mt_orig, ht_orig in zip(
#         corrected_texts, mt_texts_train, ht_sequences_train):
#     if mt_orig==corrected:
#         print(repr(mt_orig))
#         print( repr(corrected))
#         print(ht_orig)
#         print("---")
#         n_corrected+=1

In [151]:
# def has_accents(text):
#     # print(text)
#     accents = [unicodedata.name(x).find(" WITH ") > 0 for word in text for x in word]
#     return np.any(accents)
# def extract_short_words(text, n=3):
#     # print(text)
#     return [word for word in text if len(word) == n]
# info = []
# for mt_text, ht_orig in zip(mt_texts_train, ht_sequences_train):
#     mt_orig = word_regex.split(mt_text)
#     info.append({
#         "mt_accent": has_accents(mt_orig),
#         "ht_accent": has_accents(ht_orig),
#         "mt_short3": extract_short_words(mt_orig, 3),
#         "ht_short3": extract_short_words(ht_orig, 3)
#     })
#     # print(info)
#     #break
# df = pd.DataFrame(info)

In [104]:
# df[["mt_accent", "ht_accent"]].corr()

In [103]:
# train_df['accent'] = df.ht_accent

In [102]:
# train_df.groupby("CENTURY").mean()["accent"]

In [149]:

# from common import remove_caps
# df.groupby("ht_accent").mean()["mt_accent"]
# ht_words = [x for xs in df[df["ht_accent"]].ht_short3.values for x in xs 
#  if remove_cap(x) == remove_cap("μου")
# ]

In [150]:
# from collections import Counter
# sorted(Counter(ht_words).items(), key=lambda x: -x[1])

In [148]:
# Counter([x for xs in df.mt_short3.values for x in xs if remove_cap(x) == remove_cap("μου")])